In [1]:
import numpy
from urllib.request import urlopen  # Read data from a webpage like a file
import scipy.optimize  # optimization
import random

In [2]:
def parseData(fname):
  for l in open(fname):
    yield eval(l)

def parseDataFromURL(fname):
  for l in urlopen(fname):
    yield eval(l)

In [3]:
print("Reading data...")
# data = list(parseDataFromURL("http://jmcauley.ucsd.edu/cse255/data/beer/beer_50000.json"))
# Save the file to disk first...
data = list(parseData("beer_50000.json"))
print("done")

Reading data...
done


In [9]:
len(data)

50000

In [8]:
data[4]

{'beer/ABV': 7.7,
 'beer/beerId': '64883',
 'beer/brewerId': '1075',
 'beer/name': 'Cauldron DIPA',
 'beer/style': 'American Double / Imperial IPA',
 'review/appearance': 4.0,
 'review/aroma': 4.5,
 'review/overall': 4.0,
 'review/palate': 4.0,
 'review/taste': 4.5,
 'review/text': "According to the website, the style for the Caldera Cauldron changes every year. The current release is a DIPA, which frankly is the only cauldron I'm familiar with (it was an IPA/DIPA the last time I ordered a cauldron at the horsebrass several years back). In any event... at the Horse Brass yesterday.\t\tThe beer pours an orange copper color with good head retention and lacing. The nose is all hoppy IPA goodness, showcasing a huge aroma of dry citrus, pine and sandlewood. The flavor profile replicates the nose pretty closely in this West Coast all the way DIPA. This DIPA is not for the faint of heart and is a bit much even for a hophead like myslf. The finish is quite dry and hoppy, and there's barely eno

In [10]:
# Pick reviewers aged 80yo or less
data1 = [d for d in data if 'user/ageInSeconds' in d and d['user/ageInSeconds'] < 60*60*24*365*80]

In [11]:
len(data1)

10389

In [12]:
data1[0]

{'beer/ABV': 5.6,
 'beer/beerId': '12386',
 'beer/brewerId': '1075',
 'beer/name': 'Caldera OBF 15',
 'beer/style': 'American Pale Lager',
 'review/appearance': 4.0,
 'review/aroma': 3.0,
 'review/overall': 4.0,
 'review/palate': 4.0,
 'review/taste': 4.0,
 'review/text': "More of a 'dry' than a lager, tasted at the 2002 Oregon Brewers Festival. Orange color, orange flavor in nose. Light malts and fairly aggressively hopped; yet it is not very bitter. Interesting taste, complex and subtle. Light yet flavorful. Mouthfeel is full and round. Finish is clean and smooth. Aftertaste is slightly bitter. Nice beer. Would be a great beer to sip during a hot summer day.",
 'review/timeStruct': {'hour': 6,
  'isdst': 0,
  'mday': 31,
  'min': 25,
  'mon': 8,
  'sec': 23,
  'wday': 6,
  'yday': 243,
  'year': 2003},
 'review/timeUnix': 1062311123,
 'user/ageInSeconds': 1782054247,
 'user/birthdayRaw': 'Jun 23, 1958',
 'user/birthdayUnix': -363718800,
 'user/gender': 'Male',
 'user/profileName': 'b

In [13]:
# y_i = theta_0 + theta_1 x age_i
#     = [theta_0, theta_1] . [1, age_i]
def feature(datum):
  feat = [1]
  feat.append(datum['user/ageInSeconds'])
  return feat

In [18]:
X = [feature(d) for d in data1]
y = [d['review/overall'] for d in data1]

In [15]:
X[:10]

[[1, 1782054247],
 [1, 983372647],
 [1, 1263823447],
 [1, 893171047],
 [1, 958748647],
 [1, 1137506648],
 [1, 1018364647],
 [1, 1036073047],
 [1, 1239462247],
 [1, 1204383847]]

In [19]:
y[:10]

[4.0, 4.0, 4.5, 4.5, 4.5, 4.0, 4.0, 2.5, 4.5, 4.5]

In [20]:
theta,residuals,rank,s = numpy.linalg.lstsq(X, y)

/home/lukeliem/miniconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if __name__ == '__main__':


In [21]:
theta

array([ 4.09478473e+00, -1.58610537e-10])

In [33]:
# y_i = theta_0 + theta_1 x ABV
#     = [theta_0, theta_1] . [1, ABV_i]
def feature(datum):
  feat = [1]
  feat.append(datum['beer/ABV'])
  return feat

In [34]:
X = [feature(d) for d in data1]
y = [d['review/overall'] for d in data1]
theta,residuals,rank,s = numpy.linalg.lstsq(X, y)

/home/lukeliem/miniconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  app.launch_new_instance()


In [35]:
theta

array([3.4324085 , 0.06434933])

In [36]:
# MSE
residuals/len(data1)

array([0.46916258])

In [27]:
X = numpy.matrix(X)
y = numpy.matrix(y)
numpy.linalg.inv(X.T * X) * X.T * y.T

matrix([[3.4324085 ],
        [0.06434933]])

In [28]:
# y_i = theta_0 + theta_1 x ABV_i + theta_2 x ABV_i**2
#     = [theta_0, theta_1, theta_2] . [1, ABV_i, ABV_i**2]
def feature(datum):
  feat = [1]
  feat.append(datum['beer/ABV'])
  feat.append(datum['beer/ABV']**2)
  return feat

In [29]:
X = [feature(d) for d in data1]
y = [d['review/overall'] for d in data1]
theta,residuals,rank,s = numpy.linalg.lstsq(X, y)

/home/lukeliem/miniconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  app.launch_new_instance()


In [30]:
theta

array([ 3.03398592,  0.17131556, -0.00656061])

In [31]:
residuals

array([4820.9913367])

In [32]:
# MSE
residuals/len(data1)

array([0.46404768])